## Lección 2. Introducción al aprendizaje de máquina Reducción de la dimensionalidad 

### Modelos de aprendizaje de máquinas Reducción de la dimensionalidad.

### Objetivos Notebook Introducción al Aprendizaje de Máquinas Reducción de la dimensionalidad: 

1. Introducir modelos de Reducción de la dimensionalidad aplicada a la gestión de portafolios de portafolios. 

### Ejemplo Factores de riesgo estadísticos

Utilizando los siguientes datos. Encuentre los componentes principales usando los valores propios de la matriz y el método de descomposición de valores singulares y el de Minimum Linear Torsion.

| Nombre       | Descripción                                                                                        |
| ------------ | -------------------------------------------------------------------------------------------------- |
| Cash         | Índice de ICE Tresury Notes & Bills con vencimiento 0-3 meses                                      |
| TIPS         | Bonos del Tesoro de EE. UU. indexados a la inflación (TIPS), 1-5 años​                             |
| Corporativos | Bonos Corporativos con calificación crediticia AAA-A, 1-5 años​                                    |
| Emergentes   | Índice de ICE de Bonos Corporativos de Mercados Emergentes.                                        |
| MBS          | Índice de ICE de títulos respaldados por hipotecas emitidas por Agencias de EE. UU. (Agency MBS)​. |
| Equity       | Índice de retorno total del S&P500                                                                 |

In [2]:
# Cargar los datos del S&P 500.
import pandas as pd
from librerias.finance_operators import calculateLogReturns
from librerias.torsion import torsion

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
indices = pd.read_csv('Datos\\'+'Índices.csv', sep = ';')
indices

In [ ]:
indices.index = pd.to_datetime(indices['ID'])
indices = indices.drop(columns=['ID'])
indices

In [ ]:
df_returns = calculateLogReturns(indices)
df_returns = df_returns.dropna() 
covariance_matrix = df_returns.cov()
minimum_torsion_matrix = torsion(covariance_matrix, df_returns, "minimum-torsion", method='exact', max_niter=100000)

In [ ]:
# Aquí normalizamos los retornos antes de aplicarles el método de PCA.
df_returns = (df_returns - df_returns.mean())/len(df_returns) 

In [ ]:
pca_matrix1_m1 = torsion(covariance_matrix, df_returns ,"PCA", method='exact', max_niter=10000, npcomp = 6)
pca_matrix1_m2 = torsion(covariance_matrix, df_returns ,"PCA-SVD", method='exact', max_niter=10000, npcomp = 6)

In [ ]:
factor_index = ['Factor'+str(x) for x in range(1, len(df_returns.columns)+1)]
factor_loadings_mlt = pd.DataFrame(minimum_torsion_matrix, columns = covariance_matrix.columns, index =factor_index)
factor_loadings_mlt

In [ ]:
factor_loadings_pca_m1 = pd.DataFrame(pca_matrix1_m1, columns = covariance_matrix.columns, index = factor_index)
factor_loadings_pca_m2 = pd.DataFrame(pca_matrix1_m2, columns = covariance_matrix.columns, index = factor_index)


El algoritmo de Minimum Linear Torsion desarrollado por Romain Deguest et Al (2013) es un método para obtener factores estadísticos y poder desarrollar aplicaciones de Risk Budgesting o de optimización por Risk Parity. El método obtiene factores ortogonales que son mas intepretables que el modelo de PCA convencional. 

In [ ]:
ax = factor_loadings_mlt.plot.bar()
ax.set_title('Factores de Carga Método MLT')

In [ ]:
ax = factor_loadings_pca_m1.plot.bar()
ax.set_title('Factores de Carga Método PCA')

In [ ]:
ax = factor_loadings_pca_m2.plot.bar()
ax.set_title('Factores de Carga Método SVD')

### Ejemplo 3: Obtención de Factores de Nivel, Pendiente y Curva con PCA.

Una de las principales aplicaciones de PCA Y de las poco donde guarda interpretabilidad es la de Componentes Principales aplicado a la curva de rendimientos. Al aplicarlo sobre los yields, crea los componentes de nivel, pendiente y curvatura. Antes de examinar la aplicación revisemos un poco algunas características de los datos:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from librerias.extract_data import get_data 
from librerias.graphs import *

#yield_df = get_data(start_year=2010,end_year=2023)
#yield_df = yield_df.sort_index()
# yield_df.to_csv('Datos\\'+'Yields_2010-2023.csv')

In [ ]:
yield_df = pd.read_csv('Datos\\'+'Yields_2010-2023.csv')
yield_df.index = pd.to_datetime(yield_df.loc[:,'Date'])  
yield_df = yield_df.dropna()
yield_df = yield_df.iloc[:,1:]
yield_df

In [ ]:
yield_df.corr() 
### Note que la correlación es mayor cuando el vencimiento de los títulos es similar.

In [ ]:
yield_df.mean()

Se puede notar que efectivamente hay cierta prima termino en promedio a lo largo del tiempo.

In [ ]:
yield_df.diff().std()

Se muestra que a mayor plazo mayor desviación estándar.

In [ ]:

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, StandardScaler
#Step1: We calculate the mean of the scaled data.
df_normalized_mean = pd.DataFrame(scale(yield_df))
#Step2: Remove any missing data points
df_normalized_mean.dropna(inplace=True)

In [ ]:

instruments = yield_df.columns
#Step3: We calculate the PCA using fit_transform
components = ['Level', 'Slope', 'Curve']
pca = PCA(n_components=len(components))

YC_PCA = pca.fit_transform(df_normalized_mean)
#Step4: Create a SCREE plot to check the weights of each component
per_var = np.round(pca.explained_variance_ratio_ * 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
plt.bar(x=range(1, len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot')


In [ ]:
# Los eigenvalores de la matriz.

pca.explained_variance_

El 70% de la variación en las tasas se explica por el factor de nivel, el 26.7% por el factor de pendiente, el 2% por el factor de curvatura.

In [ ]:
plt.plot(labels, pca.explained_variance_ratio_.cumsum()*100)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Cumulative Explained Variance')

In [ ]:
# 
PC1, PC2, PC3 = pca.components_[0],pca.components_[1],pca.components_[2]
NumComponents = 3

topPorfolios = pd.DataFrame(pca.components_[:NumComponents],columns=instruments)
topPorfolios

In [ ]:

eigenPortfolios = topPorfolios.div(topPorfolios.sum(1),axis=0)
#eigenPortfolios.index = [ f'Portfolio {i}' for i in range(NumComponents)]
eigenPortfolios.index = ['Level', 'Slope', 'Curvature']
axs = eigenPortfolios.T.plot.bar(subplots=True,figsize=(14,10),legend=False)
plt.subplots_adjust(hspace=0.25)
axs[0].set_ylim(0,.14)

In [ ]:
# En el lenguaje de Data Science a estos Factores se les conoce como 'Scores' o PC Scores
orthogonal_factor = pd.DataFrame(YC_PCA, index = yield_df.index, columns = components)
orthogonal_factor 

In [ ]:
orthogonal_factor = orthogonal_factor.sort_index()
orthogonal_factor['Level'].plot()

In [ ]:
orthogonal_factor['Slope'].plot()

In [ ]:
orthogonal_factor['Curve'].plot()

### Referencias 

EDHEC-Risk Institute (2019). Chapter 1.2 – Technical Supplement: Scientific. Lectures Notes for Advances in Asset Allocation Seminar 2020.

EDHEC-Risk Institute (2019). Chapter 1.3 – Technical Supplement: Scientific. Lectures Notes for Advances in Asset Allocation Seminar 2020.